# using only train test split

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

c:\users\sudhe\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\sudhe\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\sudhe\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris

iris,y=load_iris(return_X_y=True)
iris_df=pd.DataFrame(iris)
iris_df['output']=y

X_train,X_test,y_train,y_test=train_test_split(iris_df.iloc[:,:-1],iris_df.iloc[:,-1])

svc=SVC(kernel='linear',C=1)
clf=svc.fit(X_train,y_train)
print(clf.score(X_test,y_test))

y_predict=svc.predict(X_test)
print(accuracy_score(y_test,y_predict))
print(pd.crosstab(y_test,y_predict))
print(classification_report(y_test,y_predict))

1.0
1.0
col_0    0   1   2
output            
0       15   0   0
1        0  12   0
2        0   0  11
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        11

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



# but here there are these hyper parameter called 'C and gamma' and those are manually set usually using grid model and so there is this risk of overfitting and there by we go for validation set

# K- cross validation

- k smaller subsets or folds

- A model is trained using k-1 of the folds as training data;

- the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

- average the metrics to get the validation metric of the complete dataset 

In [10]:
from sklearn.model_selection import cross_val_score, cross_val_predict

scores=cross_val_score(svc,X_train,y_train,cv=5)
print(scores.mean(),scores.std())
print('so accuracy is in btw {} and {}'.format(scores.mean()-2*scores.std(),min(scores.mean()+2*scores.std(),1)))
# just to get those predictions if required some time i.e. when we do find accuracy manually
y_predict_K_cv=cross_val_predict(svc,X_train,y_train,cv=5)

print(y_predict_K_cv.shape,X_train.shape)

0.9735177865612649 0.03498861023925808
so accuracy is in btw 0.9035405660827487 and 1
(112,) (112, 4)


# see now the accuracy is not one but it's 0.9735 instead.. and it can go till 0.9 if we take 2 std dev or can be low if we take the minimum value

#  cross_val_score uses the KFold or StratifiedKFold strategies by default, the latter being used if the estimator derives from ClassifierMixin.

# It is also possible to use other cross validation strategies by passing a cross validation iterator instead, for instance:

In [11]:
from sklearn.model_selection import ShuffleSplit
n_samples=X_train.shape[0]
cv=ShuffleSplit(n_splits=5,test_size=0.3,random_state=0)
cross_val_score(clf,X_train,y_train,cv=cv)

array([0.91176471, 1.        , 1.        , 0.97058824, 0.94117647])

# making a pipeline

In [16]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svc.fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)

1.0

In [18]:
from sklearn.pipeline import make_pipeline
my_pipeline=make_pipeline(preprocessing.StandardScaler(),SVC())
cross_val_score(my_pipeline,X_train,y_train,cv=cv)

array([0.94117647, 0.94117647, 0.97058824, 0.94117647, 0.91176471])

# The cross_validate function differs from cross_val_score in two ways:

# It allows specifying multiple metrics for evaluation.

# It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

In [19]:
from sklearn.model_selection import cross_validate

cross_validate(my_pipeline,X_train,y_train,cv=cv)

{'fit_time': array([0.00999308, 0.00699592, 0.00898981, 0.00899434, 0.00799584]),
 'score_time': array([0.00299859, 0.00299811, 0.00299716, 0.00299931, 0.00299788]),
 'test_score': array([0.94117647, 0.94117647, 0.97058824, 0.94117647, 0.91176471])}

# so it returns fit time, score time, test score by default and we give other metrics we want as follows

In [21]:
scoring = ['precision_macro', 'recall_macro','f1_macro']
cross_validate(my_pipeline,X_train,y_train,cv=cv,scoring=scoring)

{'fit_time': array([0.00999403, 0.00599647, 0.00599337, 0.00799561, 0.00499654]),
 'score_time': array([0.01598978, 0.00799751, 0.01799154, 0.01299143, 0.0139904 ]),
 'test_precision_macro': array([0.95238095, 0.93333333, 0.96969697, 0.93939394, 0.93333333]),
 'test_recall_macro': array([0.95238095, 0.94871795, 0.97619048, 0.94405594, 0.92857143]),
 'test_f1_macro': array([0.94871795, 0.93518519, 0.97178131, 0.94049062, 0.92296296])}

# so these two lines are what we should remember a lot
# scoring = ['precision_macro', 'recall_macro','f1_macro']
# cross_validate(my_pipeline,X_train,y_train,cv=cv,scoring=scoring)


# we can also make scorers as follows

In [23]:
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
scoring={'prec_macro':'precision_macro', 'rec_macro': make_scorer(recall_score,average='macro')}

cross_validate(my_pipeline,X_train,y_train,cv=cv,scoring=scoring)

{'fit_time': array([0.00899506, 0.00799489, 0.00799346, 0.0049932 , 0.00499725]),
 'score_time': array([0.00999427, 0.00999427, 0.01199365, 0.00599647, 0.00399828]),
 'test_prec_macro': array([0.95238095, 0.93333333, 0.96969697, 0.93939394, 0.93333333]),
 'test_rec_macro': array([0.95238095, 0.94871795, 0.97619048, 0.94405594, 0.92857143])}

# The function cross_val_predict is appropriate for:
- Visualization of predictions obtained from different models.

- Model blending: When predictions of one supervised estimator are used to train another estimator in ensemble methods.

# now there are these cross validate iterators, we used ShuffleSplit before and now we have

- KFold
- StratifiedKFold
- RepeatedKFold
- RepeatedStratifiedKFold
- LeaveOneOut
- LeavePOut

# ShuffleSplit does well and we do have StratifiedShuffleSplit

# The word stratified is used when we have each set containing approximately the same percentage of samples of each target class as the complete set.

# KFold


## KFold divides all the samples in k groups of samples, called folds (if k=n , this is equivalent to the Leave One Out strategy), of equal sizes (if possible). The prediction function is learned using k-1 folds, and the fold left out is used for test. KFold returns indices of slices and we can directly get the subset using them and we can pass all these to cross_validate

In [30]:
import numpy as np
>>> from sklearn.model_selection import KFold

>>> X = np.array(["a", "b", "c", "d"])
>>> kf = KFold(n_splits=2)
>>> for train, test in kf.split(X):
        print("%s %s" % (train, test))
        print(X[train],X[test])

[2 3] [0 1]
['c' 'd'] ['a' 'b']
[0 1] [2 3]
['a' 'b'] ['c' 'd']


# here train means k-1 folds that are trained and test means kth fold that is used for validation and this holds for every other folding strategy below

# or else send kf object to cross_validate as follows

In [31]:
cross_validate(my_pipeline,X_train,y_train,cv=KFold(5),scoring=scoring)

{'fit_time': array([0.00999308, 0.00699878, 0.00899458, 0.0089941 , 0.00899506]),
 'score_time': array([0.00899506, 0.00899291, 0.00999522, 0.00999331, 0.00899506]),
 'test_prec_macro': array([0.94444444, 0.95238095, 0.86111111, 0.95238095, 0.93333333]),
 'test_rec_macro': array([0.96969697, 0.96666667, 0.86111111, 0.96666667, 0.92592593])}

# Repeated K-Fold

## RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.

In [34]:
import numpy as np
>>> from sklearn.model_selection import RepeatedKFold
>>> X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
>>> random_state = 12883823
>>> rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=random_state)
>>> for train, test in rkf.split(X):
...     print("%s %s" % (train, test))
...     print(X[train],X[test])

[2 3] [0 1]
[[1 2]
 [3 4]] [[1 2]
 [3 4]]
[0 1] [2 3]
[[1 2]
 [3 4]] [[1 2]
 [3 4]]
[0 2] [1 3]
[[1 2]
 [1 2]] [[3 4]
 [3 4]]
[1 3] [0 2]
[[3 4]
 [3 4]] [[1 2]
 [1 2]]


In [35]:
cross_validate(my_pipeline,X_train,y_train,cv=RepeatedKFold(5,2),scoring=scoring)

{'fit_time': array([0.00799584, 0.00899553, 0.00799489, 0.00899386, 0.00499868,
        0.00599957, 0.00899267, 0.01299095, 0.00899434, 0.00799465]),
 'score_time': array([0.00999451, 0.00899553, 0.00899577, 0.00699711, 0.00799179,
        0.00699306, 0.00999475, 0.00899363, 0.01898932, 0.00499701]),
 'test_prec_macro': array([0.94444444, 1.        , 0.91666667, 0.96296296, 0.82010582,
        0.84259259, 0.87179487, 0.95238095, 1.        , 0.95238095]),
 'test_rec_macro': array([0.96969697, 1.        , 0.97222222, 0.95238095, 0.81904762,
        0.83068783, 0.84848485, 0.96296296, 1.        , 0.95238095])}

# Leave One Out (LOO)

## LeaveOneOut (or LOO) is a simple cross-validation. Each learning set is created by taking all the samples except one, the test set being the sample left out. Thus, for  samples, we have n different training sets and  different tests set. This cross-validation procedure does not waste much data as only one sample is removed from the training set:

In [39]:
from sklearn.model_selection import LeaveOneOut

X = np.array([1, 2, 3, 4])
loo = LeaveOneOut()
for train, test in loo.split(X):
    print("%s %s" % (train, test))
    print(X[train],X[test])

[1 2 3] [0]
[2 3 4] [1]
[0 2 3] [1]
[1 3 4] [2]
[0 1 3] [2]
[1 2 4] [3]
[0 1 2] [3]
[1 2 3] [4]


In [42]:
cross_validate(my_pipeline,X_train,y_train,cv=loo,scoring=scoring)

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'reca

{'fit_time': array([0.00999713, 0.01000166, 0.00899768, 0.00499725, 0.00699639,
        0.00799322, 0.00499749, 0.00499725, 0.00699568, 0.00499392,
        0.00599384, 0.0049963 , 0.00499511, 0.0059979 , 0.0070014 ,
        0.00499701, 0.00399733, 0.00600147, 0.00499773, 0.00699615,
        0.00499439, 0.00599289, 0.00499725, 0.00399804, 0.00599241,
        0.00499845, 0.00599146, 0.0039978 , 0.00599456, 0.00499845,
        0.00499797, 0.00499725, 0.00499749, 0.00499797, 0.0049963 ,
        0.00499725, 0.0049994 , 0.00499749, 0.00399756, 0.0039959 ,
        0.00799561, 0.00499725, 0.00499701, 0.00699329, 0.00699687,
        0.00499749, 0.00399756, 0.00499678, 0.00399637, 0.00399828,
        0.00599909, 0.00499749, 0.0079937 , 0.00799465, 0.00499725,
        0.0089972 , 0.00799513, 0.00899434, 0.00799465, 0.0109973 ,
        0.00499797, 0.00599885, 0.00599551, 0.00699639, 0.00499749,
        0.0059979 , 0.00499701, 0.00599647, 0.0059967 , 0.00499749,
        0.00499415, 0.0039978 , 0.00

# LOO is more computationally expensive than k-fold cross validation.

# In terms of accuracy, LOO often results in high variance as an estimator for the test error. Intuitively, since n-1 of the n samples are used to build each model, models constructed from folds are virtually identical to each other and to the model built from the entire training set.

# However, if the learning curve is steep for the training size in question, then 5- or 10- fold cross validation can overestimate the generalization error.

# <i>As a general rule, most authors, and empirical evidence, suggest that "5- or 10-" fold cross validation should be preferred to LOO.</i>

# Leave P Out (LPO)
# LeavePOut is very similar to LeaveOneOut as it creates all the possible training/test sets by removing  samples from the complete set. For n samples, this produces nCp train-test pairs. Unlike LeaveOneOut and KFold, the test sets will overlap for . <i> p>1 </i>



In [41]:
from sklearn.model_selection import LeavePOut

X = np.ones(4)
lpo = LeavePOut(p=2)
for train, test in lpo.split(X):
    print("%s %s" % (train, test))
    print(X[train],X[test])

[2 3] [0 1]
[1. 1.] [1. 1.]
[1 3] [0 2]
[1. 1.] [1. 1.]
[1 2] [0 3]
[1. 1.] [1. 1.]
[0 3] [1 2]
[1. 1.] [1. 1.]
[0 2] [1 3]
[1. 1.] [1. 1.]
[0 1] [2 3]
[1. 1.] [1. 1.]


In [43]:
cross_validate(my_pipeline,X_train,y_train,cv=lpo,scoring=scoring)

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', ave

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'reca

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', ave

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'reca

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', ave

...

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'reca

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'reca

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'reca

{'fit_time': array([0.00699639, 0.00799608, 0.00799537, ..., 0.0049963 , 0.00599718,
        0.00599837]),
 'score_time': array([0.00999355, 0.00999188, 0.00899887, ..., 0.00599575, 0.00799656,
        0.00599742]),
 'test_prec_macro': array([1., 1., 1., ..., 1., 1., 1.]),
 'test_rec_macro': array([1., 1., 1., ..., 1., 1., 1.])}

# similarly use StratifiedKFold and StratifiedShuffleSplit

# there are also group folds
- GroupKFold
- LeaveOneGroupOut
- LeavePGroupsOut
- GroupShuffleShift

# CV for Time Series Data
- Time series data is characterised by the correlation between observations that are near in time (autocorrelation). However, classical cross-validation techniques such as KFold and ShuffleSplit assume the samples are independent and identically distributed, and would result in unreasonable correlation between training and testing instances (yielding poor estimates of generalisation error) on time series data. Therefore, it is very important to evaluate our model for time series data on the “future” observations least like those that are used to train the model. To achieve this, one solution is provided by TimeSeriesSplit.

# i.e. previous CV's assume that samples are independent and identically distributed and bcoz of which a correlation btw training and testing instances on time series data so we go for specific type for TSD

# Time Series Split
- TimeSeriesSplit is a variation of k-fold which returns first <b><i>K</i></b> folds as train set and the <b><i>K+1</i></b> th fold as test set. 
# Note that unlike standard cross-validation methods, successive training sets are supersets of those that come before them. Also, it adds all surplus data to the first training partition, which is always used to train the model.

- This class can be used to cross-validate time series data samples that are observed at fixed time intervals.

In [45]:
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=3)
print(tscv)
for train, test in tscv.split(X):
    print("%s %s" % (train, test))

TimeSeriesSplit(max_train_size=None, n_splits=3)
[0 1 2] [3]
[0 1 2 3] [4]
[0 1 2 3 4] [5]


# Cross validation iterators can also be used to directly perform model selection using Grid Search for the optimal hyperparameters of the model. and use <i>"GridSearchCV" of sklearn.model_selection</i>

# as cross_validate, GridSearchCV requires estimator, scoring, folds/subsets(i.e. cv parameter) and mainly <i>"grid of tuning hyperparameters"</i> and it is shown as follows

In [59]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

clf=GridSearchCV(svc, parameters,cv=cv)
clf.fit(X_train,y_train)
print(clf.cv_results_)
print("RESULTS")
print(clf.cv_results_.keys())
# since there are 5 splits or folds in cv we get these
# and there are 4 possible combinations from grid so we get 4 in each split
print(clf.cv_results_['split0_test_score'])
print(clf.cv_results_['split1_test_score'])
print(clf.cv_results_['split2_test_score'])
print(clf.cv_results_['split3_test_score'])
print(clf.cv_results_['split4_test_score'])

# so we chose as follows


{'mean_fit_time': array([0.00499678, 0.00539627, 0.00419774, 0.00399776]), 'std_fit_time': array([0.00126354, 0.00079982, 0.0009793 , 0.00126407]), 'mean_score_time': array([0.00219903, 0.00219855, 0.00219879, 0.00219874]), 'std_score_time': array([0.00097965, 0.00074798, 0.00040038, 0.00074844]), 'param_C': masked_array(data=[1, 1, 10, 10],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 1, 'kernel': 'linear'}, {'C': 1, 'kernel': 'rbf'}, {'C': 10, 'kernel': 'linear'}, {'C': 10, 'kernel': 'rbf'}], 'split0_test_score': array([0.91176471, 0.85294118, 0.91176471, 0.91176471]), 'split1_test_score': array([1.        , 0.97058824, 0.97058824, 0.97058824]), 'split2_test_score': array([1.        , 0.94117647, 0.97058824, 1.        ]), 'split3_test_score': array([

c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\sudhe\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureW

# since there are 5 splits or folds in cv we get these
# and there are 4 possible combinations from grid so we get 4 in each split


In [60]:
results=[clf.cv_results_['split0_test_score'],clf.cv_results_['split1_test_score'],clf.cv_results_['split2_test_score'],clf.cv_results_['split3_test_score'],clf.cv_results_['split4_test_score']]

In [61]:
results=np.array(results)

In [62]:
results.shape

(5, 4)

In [63]:
df=pd.DataFrame(results)

In [64]:
df

,0,1,2,3
0,0.911765,0.852941,0.911765,0.911765
1,1.000000,0.970588,0.970588,0.970588
2,1.000000,0.941176,0.970588,1.000000
3,0.970588,0.941176,0.941176,0.970588
4,0.941176,0.882353,0.970588,0.970588


In [70]:
optimum_hyper_parameters_index=np.argmax(list(map(lambda x: df.iloc[:,x].mean(),np.arange(4))))

In [71]:
optimum_hyper_parameters_index

0

In [73]:
clf.cv_results_['params'][optimum_hyper_parameters_index]

{'C': 1, 'kernel': 'linear'}

# so best hyper parameters are c=1 and kernel is linear